In [1]:
from sklearn.preprocessing import MinMaxScaler , StandardScaler , QuantileTransformer
from sklearn.metrics import r2_score
from keras.layers import LSTM, Dense, Dropout, Input, Concatenate
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras import metrics, regularizers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
epochs = 3000
shared_units = Volume_units = ema6_units = ema12_units = ema26_units = macd_units = wrsi_units = K_units = D_units = closed_units = 16
batch_size = 64

early_stop = EarlyStopping(monitor='dense_9_loss', patience=20)

def splitData(X,Y,rate):
    X_train = X[:int(X.shape[0]*rate)]
    Y_train = Y[:int(Y.shape[0]*rate)]
    X_val = X[int(X.shape[0]*rate):]
    Y_val = Y[int(Y.shape[0]*rate)-1:-1]
    return X_train, Y_train, X_val, Y_val

df = pd.read_csv("data/2330_indicators.csv")
df = df.drop('Unnamed: 0',axis=1)
print(df)

X_train = df.values
y_train = df.loc[1:].values
X_train , Y_train , X_val , Y_val = splitData(X_train,y_train,0.8)
shared_lstm_init = LSTM(shared_units,activation='relu',kernel_initializer='lecun_uniform',kernel_regularizer = regularizers.l2(0.0001),return_sequences = True)
shared_lstm1 = LSTM(shared_units,kernel_regularizer=regularizers.l2(0.0001),return_sequences=True)
shared_lstm2 = LSTM(shared_units,kernel_regularizer=regularizers.l2(0.0001),return_sequences=True)
shared_lstm3 = LSTM(shared_units,kernel_regularizer=regularizers.l2(0.0001),return_sequences=False)

      closed      volume        ema6       ema12       ema26      macd  \
0       96.0      4000.0    0.000000    0.000000    0.000000  0.000000   
1      102.5      1000.0    0.000000    0.000000    0.000000  0.000000   
2      109.5      1000.0    0.000000    0.000000    0.000000  0.000000   
3      117.0      2000.0    0.000000    0.000000    0.000000  0.000000   
4      125.0      1000.0    0.000000    0.000000    0.000000  0.000000   
5      133.5     37016.0  113.916667    0.000000    0.000000  0.000000   
6      142.5   3855000.0  123.444444    0.000000    0.000000  0.000000   
7      152.0  12541527.0  132.962963    0.000000    0.000000  0.000000   
8      162.0  10301426.0  142.641975    0.000000    0.000000  0.000000   
9      173.0  17770605.0  152.094650    0.000000    0.000000  0.000000   
10     171.0  11219138.0  158.646433    0.000000    0.000000  0.000000   
11     171.0   8139580.0  162.930956  137.854167    0.000000  0.000000   
12     173.0   6979000.0  165.953970  

Normalization

In [3]:
sc_X = QuantileTransformer()
#X_train_sc = sc_X.fit_transform(X_train)
#X_val_sc = sc_X.transform(X_val)
X_train_df = pd.DataFrame(data=X_train)
X_val_df = pd.DataFrame(data=X_val)
sc = MinMaxScaler(feature_range=(np.min(df.loc[:,'closed'].values),np.max(df.loc[:,'closed'].values)))
a = np.array(X_train_df.loc[:,1].values)
a = a.reshape(a.shape[0],1)
b = np.array(X_val_df.loc[:,1].values)
b = b.reshape(b.shape[0],1)
a = sc.fit_transform(a)
b = sc.transform(b)

Layer#1 Volume Training

In [4]:
Volume_X_train = a.reshape(a.shape[0])
Volume_Y_train = Volume_X_train[1:]
Volume_X_train = Volume_X_train[:-1]
Volume_X_val = b.reshape(b.shape[0])
Volume_Y_val = Volume_X_val[1:]
Volume_X_val = Volume_X_val[:-1]
Volume_X_train = Volume_X_train.reshape(Volume_X_train.shape[0],1,1)
Volume_X_val = Volume_X_val.reshape(Volume_X_val.shape[0],1,1)
VolumeInput = Input(shape=(1,1))
Volume_model = shared_lstm_init(VolumeInput)
Volume_model = shared_lstm1(Volume_model)
Volume_model = shared_lstm2(Volume_model)
Volume_out = shared_lstm3(Volume_model)
Volume_model = Dense(1)(Volume_out)

Layer#2 ema6

In [ ]:
ema6_X_train = X_train_df.loc[:,2].values 
ema6_Y_train = ema6_X_train[1:] 
ema6_X_train = ema6_X_train[:-1]
ema6_X_val = X_val_df.loc[:,2].values
ema6_Y_val = ema6_X_val[1:]
ema6_X_val = ema6_X_val[:-1]
ema6_X_train = ema6_X_train.reshape(ema6_X_train.shape[0],1,1)
ema6_X_val = ema6_X_val.reshape(ema6_X_val.shape[0],1,1)
ema6Input = Input(shape=(1,1))
ema6_model = shared_lstm_init(ema6Input)
ema6_model = shared_lstm1(ema6_model)
ema6_model = shared_lstm2(ema6_model)
ema6_out = shared_lstm3(ema6_model)
ema6_model = Dense(1)(ema6_out)

In [ ]:
"""
model = Model(inputs=[VolumeInput,ema6Input],outputs=[Volume_model,ema6_model])
model.compile(loss = 'mean_squared_error',optimizer='Adam')
model.fit([Volume_X_train,ema6_X_train],[Volume_Y_train,ema6_Y_train],epochs=epochs,batch_size=batch_size,shuffle=False,callbacks=[early_stop])
#model.fit([Volume_X_train,ema6_X_train],[Volume_Y_train,ema6_Y_train],epochs=epochs,batch_size=batch_size,shuffle=False,callbacks=[early_stop])
preds = model.predict([Volume_X_val,ema6_X_val])
r2score = r2_score(Volume_Y_val,preds[0])
plt.plot(preds[0][:100])
plt.plot(Volume_Y_val[:100])
plt.show()
print("The R2 score on the Validation set is:\t{:0.3f}".format(r2score))
"""

'\nmodel = Model(inputs=[VolumeInput,ema6Input],outputs=[Volume_model,ema6_model])\nmodel.compile(loss = \'mean_squared_error\',optimizer=\'Adam\')\nmodel.fit([Volume_X_train,ema6_X_train],[Volume_Y_train,ema6_Y_train],epochs=epochs,batch_size=batch_size,shuffle=False,callbacks=[early_stop])\n#model.fit([Volume_X_train,ema6_X_train],[Volume_Y_train,ema6_Y_train],epochs=epochs,batch_size=batch_size,shuffle=False,callbacks=[early_stop])\npreds = model.predict([Volume_X_val,ema6_X_val])\nr2score = r2_score(Volume_Y_val,preds[0])\nplt.plot(preds[0][:100])\nplt.plot(Volume_Y_val[:100])\nplt.show()\nprint("The R2 score on the Validation set is:\t{:0.3f}".format(r2score))\n'

Layer#2 ema12

In [ ]:
ema12_X_train = X_train_df.loc[:,3].values
ema12_Y_train = ema12_X_train[1:] 
ema12_X_train = ema12_X_train[:-1]
ema12_X_val = X_val_df.loc[:,3].values 
ema12_Y_val = ema12_X_val[1:]
ema12_X_val = ema12_X_val[:-1]
ema12_X_train = ema12_X_train.reshape(ema12_X_train.shape[0],1,1)
ema12_X_val = ema12_X_val.reshape(ema12_X_val.shape[0],1,1)
ema12Input = Input(shape=(1,1))
ema12_model = shared_lstm_init(ema12Input)
ema12_model = shared_lstm1(ema12_model)
ema12_model = shared_lstm2(ema12_model)
ema12_out = shared_lstm3(ema12_model)
ema12_model = Dense(1,activation='relu')(ema12_out)
print(ema12_X_val.shape,ema12_Y_val.shape)

(1257, 1, 1) (1257,)


In [ ]:
ema26_X_train = X_train_df.loc[:,4].values 
ema26_Y_train = ema26_X_train[1:] 
ema26_X_train = ema26_X_train[:-1]
ema26_X_val = X_val_df.loc[:,4].values
ema26_Y_val = ema26_X_val[1:]
ema26_X_val = ema26_X_val[:-1]
ema26_X_train = ema26_X_train.reshape(ema26_X_train.shape[0],1,1)
ema26_X_val = ema26_X_val.reshape(ema26_X_val.shape[0],1,1)
ema26Input = Input(shape=(1,1))
ema26_model = shared_lstm_init(ema26Input)
ema26_model = shared_lstm1(ema26_model)
ema26_model = shared_lstm2(ema26_model)
ema26_out = shared_lstm3(ema26_model)
ema26_model = Dense(1,activation='relu')(ema26_out)
print(ema26_X_val.shape,ema26_Y_val.shape)

(1257, 1, 1) (1257,)


In [ ]:
macd_X_train = X_train_df.loc[:,5].values 
macd_Y_train = macd_X_train[1:] 
macd_X_train = macd_X_train[:-1]
macd_X_val = X_val_df.loc[:,5].values 
macd_Y_val = macd_X_val[1:]
macd_X_val = macd_X_val[:-1]
macd_X_train = macd_X_train.reshape(macd_X_train.shape[0],1,1)
macd_X_val = macd_X_val.reshape(macd_X_val.shape[0],1,1)
macdInput = Input(shape=(1,1))
macd_model = shared_lstm_init(macdInput)
macd_model = shared_lstm1(macd_model)
macd_model = shared_lstm2(macd_model)
macd_out = shared_lstm3(macd_model)
macd_model = Dense(1,activation='relu')(macd_out)

In [ ]:
wrsi_X_train = X_train_df.loc[:,6].values 
wrsi_Y_train = wrsi_X_train[1:] 
wrsi_X_train = wrsi_X_train[:-1]
wrsi_X_val = X_val_df.loc[:,6].values 
wrsi_Y_val = wrsi_X_val[1:]
wrsi_X_val = wrsi_X_val[:-1]
wrsi_X_train = wrsi_X_train.reshape(wrsi_X_train.shape[0],1,1)
wrsi_X_val = wrsi_X_val.reshape(wrsi_X_val.shape[0],1,1)
wrsiInput = Input(shape=(1,1))
wrsi_model = shared_lstm_init(wrsiInput)
wrsi_model = shared_lstm1(wrsi_model)
wrsi_model = shared_lstm2(wrsi_model)
wrsi_out = shared_lstm3(wrsi_model)
wrsi_model = Dense(1,activation='relu')(wrsi_out)
print(wrsi_X_val.shape,wrsi_Y_val.shape)

(1257, 1, 1) (1257,)


In [ ]:
K_X_train = X_train_df.loc[:,7].values
K_Y_train = K_X_train[1:] 
K_X_train = K_X_train[:-1]
K_X_val = X_val_df.loc[:,7].values
K_Y_val = K_X_val[1:]
K_X_val = K_X_val[:-1]
K_X_train = K_X_train.reshape(K_X_train.shape[0],1,1)
K_X_val = K_X_val.reshape(K_X_val.shape[0],1,1)
KInput = Input(shape=(1,1))
K_model = shared_lstm_init(KInput)
K_model = shared_lstm1(K_model)
K_model = shared_lstm2(K_model)
K_out = shared_lstm3(K_model)
K_model = Dense(1,activation='relu')(K_out)
print(K_X_val.shape,K_Y_val.shape)

(1257, 1, 1) (1257,)


In [ ]:
D_X_train = X_train_df.loc[:,8].values
D_Y_train = D_X_train[1:] 
D_X_train = D_X_train[:-1]
D_X_val = X_val_df.loc[:,8].values 
D_Y_val = D_X_val[1:]
D_X_val = D_X_val[:-1]
D_X_train = D_X_train.reshape(D_X_train.shape[0],1,1)
D_X_val = D_X_val.reshape(D_X_val.shape[0],1,1)
DInput = Input(shape=(1,1))
D_model = shared_lstm_init(DInput)
D_model = shared_lstm1(D_model)
D_model = shared_lstm2(D_model)
D_out = shared_lstm3(D_model)
D_model = Dense(1,activation='relu')(D_out)
print(D_X_val.shape,D_Y_val.shape)

(1257, 1, 1) (1257,)


In [ ]:
closed_X_train = X_train_df.loc[:,0].values 
closed_Y_train = closed_X_train[1:] 
closed_X_train = closed_X_train[:-1]
closed_X_val = X_val_df.loc[:,0].values 
closed_Y_val = closed_X_val[1:]
closed_X_val = closed_X_val[:-1]
closed_X_train = closed_X_train.reshape(closed_X_train.shape[0],1,1)
closed_X_val = closed_X_val.reshape(closed_X_val.shape[0],1,1)
closedInput = Input(shape=(1,1))
#closed_model = Concatenate([Volume_out,ema6_out,ema12_out,ema26_out,macd_out,wrsi_out,K_out,D_out,closedInput])
closed_model = shared_lstm_init(closedInput)
closed_model = shared_lstm1(closed_model)
closed_model = shared_lstm2(closed_model)
closed_out = shared_lstm3(closed_model)
closed_model = Dense(1,activation='relu')(closed_out)
merged_model = Model(inputs=[VolumeInput,ema6Input,ema12Input,ema26Input,macdInput,wrsiInput,KInput,DInput,closedInput],outputs=[Volume_model,ema6_model,ema12_model,ema26_model,macd_model,wrsi_model,K_model,D_model,closed_model])
merged_model.summary()
merged_model.compile(loss='mean_squared_error',optimizer='Adam')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1, 1)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1, 1)         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1, 1)         0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 1, 1)         0                                            
__________________________________________________________________________________________________
input_5 (I

In [ ]:

history = merged_model.fit([Volume_X_train,ema6_X_train,ema12_X_train,ema26_X_train,macd_X_train,wrsi_X_train,K_X_train,D_X_train,closed_X_train],[Volume_Y_train,ema6_Y_train,ema12_Y_train,ema26_Y_train,macd_Y_train,wrsi_Y_train,K_Y_train,D_Y_train,closed_Y_train],epochs=epochs,batch_size=batch_size,shuffle=False,validation_data=([Volume_X_val,ema6_X_val,ema12_X_val,ema26_X_val,macd_X_val,wrsi_X_val,K_X_val,D_X_val,closed_X_val],[Volume_Y_val,ema6_Y_val,ema12_Y_val,ema26_Y_val,macd_Y_val,wrsi_Y_val,K_Y_val,D_Y_val,closed_Y_val]),callbacks=[early_stop])

Train on 5028 samples, validate on 1257 samples
Epoch 1/3000
5028/5028 [==============================] - 37s 7ms/step - loss: 43471.4551 - dense_1_loss: 1651.0426 - dense_2_loss: 8157.4089 - dense_3_loss: 8142.8785 - dense_4_loss: 8046.1937 - dense_5_loss: 7.6041 - dense_6_loss: 3016.5006 - dense_7_loss: 3198.5249 - dense_8_loss: 3079.7379 - dense_9_loss: 8171.5514 - val_loss: 142991.9808 - val_dense_1_loss: 1553.5909 - val_dense_2_loss: 32802.8514 - val_dense_3_loss: 32792.7938 - val_dense_4_loss: 32454.4444 - val_dense_5_loss: 5.3196 - val_dense_6_loss: 3217.3971 - val_dense_7_loss: 3697.1029 - val_dense_8_loss: 3625.1337 - val_dense_9_loss: 32843.3430
Epoch 2/3000
5028/5028 [==============================] - 7s 1ms/step - loss: 42045.9651 - dense_1_loss: 1551.0531 - dense_2_loss: 7871.6361 - dense_3_loss: 7986.0887 - dense_4_loss: 7763.1481 - dense_5_loss: 7.6037 - dense_6_loss: 3016.5159 - dense_7_loss: 2961.7737 - dense_8_loss: 3079.7379 - dense_9_loss: 7808.3951 - val_loss: 1369

5028/5028 [==============================] - 7s 1ms/step - loss: 31043.4603 - dense_1_loss: 624.7132 - dense_2_loss: 5623.0526 - dense_3_loss: 5915.0770 - dense_4_loss: 5594.1066 - dense_5_loss: 7.6037 - dense_6_loss: 3016.5159 - dense_7_loss: 1694.0412 - dense_8_loss: 3079.7379 - dense_9_loss: 5488.5959 - val_loss: 117949.5116 - val_dense_1_loss: 547.4227 - val_dense_2_loss: 27075.8200 - val_dense_3_loss: 27741.9233 - val_dense_4_loss: 26931.0185 - val_dense_5_loss: 5.3196 - val_dense_6_loss: 3217.3971 - val_dense_7_loss: 2015.0206 - val_dense_8_loss: 3625.1337 - val_dense_9_loss: 26790.4405nse_2_loss: 6458.1084 - dense_3_loss: 6763.6156 - dense_4_loss: 6414.2490 - dense_5_loss: 10.0948 - dense_6_loss: 2978.4976 - dense_7_loss: 1633.2655 - dense_8_loss: 2964.0737 - 
Epoch 15/3000
5028/5028 [==============================] - 7s 1ms/step - loss: 30465.7733 - dense_1_loss: 582.0544 - dense_2_loss: 5505.2599 - dense_3_loss: 5791.9218 - dense_4_loss: 5475.7953 - dense_5_loss: 7.6037 - dens

5028/5028 [==============================] - 8s 2ms/step - loss: 24620.7068 - dense_1_loss: 227.1335 - dense_2_loss: 4285.0039 - dense_3_loss: 4511.9417 - dense_4_loss: 4249.9655 - dense_5_loss: 7.6037 - dense_6_loss: 3016.5159 - dense_7_loss: 1061.3168 - dense_8_loss: 3079.7379 - dense_9_loss: 4181.4702 - val_loss: 103318.6478 - val_dense_1_loss: 177.7356 - val_dense_2_loss: 23705.4264 - val_dense_3_loss: 24285.1092 - val_dense_4_loss: 23555.6437 - val_dense_5_loss: 5.3196 - val_dense_6_loss: 3217.3971 - val_dense_7_loss: 1286.7470 - val_dense_8_loss: 3625.1337 - val_dense_9_loss: 23460.1218
Epoch 28/3000
5028/5028 [==============================] - 8s 2ms/step - loss: 24210.6005 - dense_1_loss: 208.1916 - dense_2_loss: 4197.3338 - dense_3_loss: 4419.5985 - dense_4_loss: 4161.8533 - dense_5_loss: 7.6037 - dense_6_loss: 3016.5159 - dense_7_loss: 1023.7250 - dense_8_loss: 3079.7379 - dense_9_loss: 4096.0237 - val_loss: 102311.2095 - val_dense_1_loss: 160.5119 - val_dense_2_loss: 23469.8

5028/5028 [==============================] - 8s 2ms/step - loss: 20033.2122 - dense_1_loss: 68.1619 - dense_2_loss: 3286.5734 - dense_3_loss: 3455.9356 - dense_4_loss: 3246.2132 - dense_5_loss: 7.6037 - dense_6_loss: 3016.5159 - dense_7_loss: 662.0365 - dense_8_loss: 3079.7379 - dense_9_loss: 3210.4161 - val_loss: 91298.8111 - val_dense_1_loss: 36.9501 - val_dense_2_loss: 20865.8006 - val_dense_3_loss: 21361.6882 - val_dense_4_loss: 20711.3447 - val_dense_5_loss: 5.3196 - val_dense_6_loss: 3217.3971 - val_dense_7_loss: 815.0853 - val_dense_8_loss: 3625.1337 - val_dense_9_loss: 20660.0752
Epoch 41/3000
5028/5028 [==============================] - 8s 2ms/step - loss: 19741.0050 - dense_1_loss: 62.1530 - dense_2_loss: 3221.5731 - dense_3_loss: 3386.7909 - dense_4_loss: 3180.8448 - dense_5_loss: 7.6037 - dense_6_loss: 3016.5159 - dense_7_loss: 638.3867 - dense_8_loss: 3079.7379 - dense_9_loss: 3147.3809 - val_loss: 90463.9959 - val_dense_1_loss: 31.9644 - val_dense_2_loss: 20666.4213 - val

5028/5028 [==============================] - 8s 2ms/step - loss: 16796.0725 - dense_1_loss: 26.8529 - dense_2_loss: 2556.6975 - dense_3_loss: 2675.5660 - dense_4_loss: 2512.1132 - dense_5_loss: 7.6037 - dense_6_loss: 3016.5159 - dense_7_loss: 416.6483 - dense_8_loss: 3079.7379 - dense_9_loss: 2504.3187 - val_loss: 81342.0923 - val_dense_1_loss: 5.5855 - val_dense_2_loss: 18471.3860 - val_dense_3_loss: 18883.9633 - val_dense_4_loss: 18312.4309 - val_dense_5_loss: 5.3196 - val_dense_6_loss: 3217.3971 - val_dense_7_loss: 515.8527 - val_dense_8_loss: 3625.1337 - val_dense_9_loss: 18305.0067
Epoch 54/3000
5028/5028 [==============================] - 8s 2ms/step - loss: 16593.2070 - dense_1_loss: 25.8293 - dense_2_loss: 2510.2331 - dense_3_loss: 2625.5413 - dense_4_loss: 2465.3826 - dense_5_loss: 7.6037 - dense_6_loss: 3016.5159 - dense_7_loss: 402.8413 - dense_8_loss: 3079.7379 - dense_9_loss: 2459.5035 - val_loss: 80652.9259 - val_dense_1_loss: 5.2765 - val_dense_2_loss: 18304.4944 - val_d

...

5028/5028 [==============================] - 8s 2ms/step - loss: 5495.3278 - dense_1_loss: 22.4991 - dense_2_loss: 407.6918 - dense_3_loss: 407.2077 - dense_4_loss: 379.6071 - dense_5_loss: 7.6037 - dense_6_loss: 670.1790 - dense_7_loss: 105.0551 - dense_8_loss: 3079.7379 - dense_9_loss: 415.6997 - val_loss: 29570.2950 - val_dense_1_loss: 5.6490 - val_dense_2_loss: 6301.6964 - val_dense_3_loss: 6341.5773 - val_dense_4_loss: 6176.4077 - val_dense_5_loss: 5.3196 - val_dense_6_loss: 674.1880 - val_dense_7_loss: 94.1542 - val_dense_8_loss: 3625.1337 - val_dense_9_loss: 6346.1217- dense_2_loss: 658.4249 - dense_3_loss: 657.3324 - dense_4_loss: 612.9848 - dense_5_loss: 11.8539 - dense_6_loss: 687.4329 - dense_7_loss: 112.9770 - dense_8_loss: 2954.2680 - dense_9_loss: 670.55 - ETA: 2s - loss: 6302.5959 - dense_1_loss: 27.1427 - dense_2_loss: 645.0330 - dense_3_loss: 643.9785 - dense_4_loss: 600.5151 - dense_5_loss: 11.6469 - dense_6_loss: 680.6604 - dense_7_loss: 112.1359 - dense_8_l
Epoch 19

5028/5028 [==============================] - 8s 2ms/step - loss: 4870.7343 - dense_1_loss: 21.7127 - dense_2_loss: 328.1868 - dense_3_loss: 329.7394 - dense_4_loss: 303.4072 - dense_5_loss: 7.6037 - dense_6_loss: 387.4758 - dense_7_loss: 76.3655 - dense_8_loss: 3079.7379 - dense_9_loss: 336.4569 - val_loss: 27329.7078 - val_dense_1_loss: 6.9254 - val_dense_2_loss: 5807.3739 - val_dense_3_loss: 5861.6245 - val_dense_4_loss: 5683.2845 - val_dense_5_loss: 5.3196 - val_dense_6_loss: 386.9729 - val_dense_7_loss: 90.5721 - val_dense_8_loss: 3625.1337 - val_dense_9_loss: 5862.4526ss: 9.5729 - dense_2_loss: 622.5384 - dense_3_loss: 620.7192 - dense_4_loss: 554.0732 - dense_5_loss: 17.6773 - dense_6_loss: 439.4505 - dense_7_loss: 75.6399 - dense_8_loss: 2917.5718 - dense_9_loss: 640. - ETA: 4s - loss: 6658.7890 - dense_1_loss: 8.8293 - dense_2_loss: 801.7273 - dense_3_loss: 789.3553 - dense_4_loss: 684.5052 - dense_5_loss: 18.0491 
Epoch 207/3000
5028/5028 [==============================] - 8s 

5028/5028 [==============================] - 8s 2ms/step - loss: 4479.9085 - dense_1_loss: 19.6487 - dense_2_loss: 269.8497 - dense_3_loss: 271.9667 - dense_4_loss: 246.9096 - dense_5_loss: 7.6037 - dense_6_loss: 237.2029 - dense_7_loss: 68.5013 - dense_8_loss: 3079.7379 - dense_9_loss: 278.4377 - val_loss: 25177.7442 - val_dense_1_loss: 4.1849 - val_dense_2_loss: 5308.3338 - val_dense_3_loss: 5368.9484 - val_dense_4_loss: 5188.8512 - val_dense_5_loss: 5.3196 - val_dense_6_loss: 234.8177 - val_dense_7_loss: 73.8127 - val_dense_8_loss: 3625.1337 - val_dense_9_loss: 5368.2923
Epoch 220/3000
5028/5028 [==============================] - 9s 2ms/step - loss: 4457.9632 - dense_1_loss: 19.4827 - dense_2_loss: 266.3435 - dense_3_loss: 268.4365 - dense_4_loss: 243.4621 - dense_5_loss: 7.6037 - dense_6_loss: 229.6754 - dense_7_loss: 68.3029 - dense_8_loss: 3079.7379 - dense_9_loss: 274.8683 - val_loss: 25022.0329 - val_dense_1_loss: 4.0823 - val_dense_2_loss: 5271.1225 - val_dense_3_loss: 5332.06

Epoch 233/3000
5028/5028 [==============================] - 8s 2ms/step - loss: 4222.7000 - dense_1_loss: 19.1020 - dense_2_loss: 224.2640 - dense_3_loss: 226.3595 - dense_4_loss: 203.8183 - dense_5_loss: 7.6037 - dense_6_loss: 161.3718 - dense_7_loss: 67.9332 - dense_8_loss: 3079.7379 - dense_9_loss: 232.4591 - val_loss: 23116.1782 - val_dense_1_loss: 4.3861 - val_dense_2_loss: 4809.2726 - val_dense_3_loss: 4871.4160 - val_dense_4_loss: 4699.7456 - val_dense_5_loss: 5.3196 - val_dense_6_loss: 157.1296 - val_dense_7_loss: 74.0406 - val_dense_8_loss: 3625.1337 - val_dense_9_loss: 4869.6838
Epoch 234/3000
5028/5028 [==============================] - 8s 2ms/step - loss: 4208.7633 - dense_1_loss: 19.1598 - dense_2_loss: 221.6586 - dense_3_loss: 223.7339 - dense_4_loss: 201.3278 - dense_5_loss: 7.6037 - dense_6_loss: 157.1618 - dense_7_loss: 68.4599 - dense_8_loss: 3079.7379 - dense_9_loss: 229.8693 - val_loss: 22978.6585 - val_dense_1_loss: 4.0577 - val_dense_2_loss: 4775.5138 - val_dense_

Epoch 247/3000
5028/5028 [==============================] - 8s 2ms/step - loss: 4039.2461 - dense_1_loss: 19.9169 - dense_2_loss: 186.1553 - dense_3_loss: 188.0713 - dense_4_loss: 168.5089 - dense_5_loss: 7.6037 - dense_6_loss: 126.8388 - dense_7_loss: 68.3507 - dense_8_loss: 3079.7379 - dense_9_loss: 194.0107 - val_loss: 21295.9162 - val_dense_1_loss: 4.1814 - val_dense_2_loss: 4360.1320 - val_dense_3_loss: 4419.7260 - val_dense_4_loss: 4262.6545 - val_dense_5_loss: 5.3196 - val_dense_6_loss: 125.5977 - val_dense_7_loss: 73.9539 - val_dense_8_loss: 3625.1337 - val_dense_9_loss: 4419.1653
Epoch 248/3000
5028/5028 [==============================] - 8s 2ms/step - loss: 4023.8581 - dense_1_loss: 19.4421 - dense_2_loss: 183.0929 - dense_3_loss: 184.9551 - dense_4_loss: 165.3543 - dense_5_loss: 7.6037 - dense_6_loss: 124.8738 - dense_7_loss: 67.8108 - dense_8_loss: 3079.7379 - dense_9_loss: 190.9355 - val_loss: 21172.1477 - val_dense_1_loss: 4.1019 - val_dense_2_loss: 4329.3637 - val_dense_

In [ ]:
#print(closed_X_val.shape,Volume_X_val.shape,ema6_X_val.shape,ema12_X_val.shape,ema26_X_val.shape,macd_X_val.shape,wrsi_X_val.shape,K_X_val.shape,D_X_val.shape)
preds = merged_model.predict([Volume_X_val,ema6_X_val,ema12_X_val,ema26_X_val,macd_X_val,wrsi_X_val,K_X_val,D_X_val,closed_X_val])
r2score = r2_score(Volume_Y_val,preds[0])
print("The R2 score on the Validation set is:\t{:0.3f}".format(r2score))
plt.plot(preds[8])
plt.plot(closed_Y_val)
plt.show()

In [ ]:
his = history.history
for k in his.keys():
    plt.title(k)
    plt.plot(his[k])
    plt.show()